In [10]:
import pandas as pd

from openai import AsyncOpenAI
import asyncio
from tqdm.asyncio import tqdm
import tiktoken

import os
from dotenv import load_dotenv

import pprint

In [ ]:
import sys
module_path = "../../climateguard"

if module_path not in sys.path:
    sys.path.append(module_path)

import clean_text
from clean_text import run_correction

In [14]:
f = "../../../data/raw/18_channels_2023_09_to_2024_09.parquet"
df = pd.read_parquet(f)

In [32]:
df.shape

(185738, 10)

In [37]:
df.head()

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens
id,,,,,,,,,,
84e93fcd88fa86338651973326536d27d29f724a4864350d7c234501ea0813d0,2023-09-01 19:40:00,choses le combat d' andré boma et du parc des ...,arte,False,Information - Journal,JT,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""\u00e9nergies fossiles"", ""timest...",3,448
21037a5700f15f8b1fb968dcf35c047bd0933816bc3f01b339be2af568983010,2023-09-01 19:42:00,vient d' engager cinq milliards d' euros dans ...,arte,False,Information - Journal,JT,"[""changement_climatique_causes"", ""attenuation_...","[{""keyword"": ""\u00e9nergie renouvelable"", ""tim...",1,408
a3aceee186595fab3603853ea38f3c51fe9453912a280ccbf0e669fda086940e,2023-09-01 20:20:00,parler une révolution par contre au niger euh ...,arte,False,Information - Magazine,28 minutes,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""climatique"", ""timestamp"": 169359...",1,647
d6b08b996cea24381de86c31a243122baa83bf750e03c91fe6530a401fc0527a,2023-09-01 20:44:00,il faut rappeler le début de cette affaire c' ...,arte,False,Information - Magazine,28 minutes,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""vivant"", ""timestamp"": 1693593844...",1,537
391397ace2825ad3d4169797e37b685d1be499dd7ccea33d89ec3e757392f243,2023-09-02 19:40:00,grandes plantations agricoles ont également mi...,arte,False,Information - Journal,JT,"[""biodiversite_concepts_generaux"", ""biodiversi...","[{""keyword"": ""agricole"", ""timestamp"": 16936763...",6,443


In [36]:
pprint.pprint(df.iloc[0].text)

("choses le combat d' andré boma et du parc des virunga pour protéger les gori "
 "a fait l' objet d' un documentaire produit par leonardo dicaprio et nommé "
 'aux oscars en deux mille quinze on voit ici le soigneur sur le tapis rouge '
 "dans la région l' exploitation illégale des ressources et les affrontements "
 'armés détruisent la biodiversité par ailleurs en deux mille vingt vingt et '
 'un travailleur du parc ont été tués le dessous des images a bien tenté de '
 "joindre andré boma pour l' interviewer mais le personnel avait été évacué "
 "fin octobre octobre deux mille vingt-deux suite à l' arrivée d' un groupe "
 "rebelle à l' intérieur du parc seuls quelques eco-gardes maintenait une "
 "présence à l' orphelinat des gori ainsi cette photo virale traduit elle la "
 'vulnérabilité des grands singes mais aussi des humains cette enquête et '
 'signé julia montfort pour le dessous des images salut à tous <unk> sonia '
 'devillers et à lundi un bien sûr alors dans une semaine tou

In [15]:
correction = run_correction(df.iloc[:1].text)

In [17]:
print(correction)

<coroutine object run_correction at 0x120cf88c0>
